# core

> Compose analytics modules for Wodan & AICore using NBDev literate programming tool.

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *  # noqa: F403


In [ ]:
#| export
import datetime
import pytz
import logging

from numpy import datetime64, timedelta64

from dateutil import parser
from functools import lru_cache


## Logging

In [ ]:
#| export
def init_console_logging(name, level=logging.INFO, timestamp=True, formatstring=""):
    '''Setup none-blocking stream handler for sending loggin to the console.'''

    # Only if no handlers defined.
    if not logging.getLogger(name).handlers:

        logger = logging.getLogger()
        logger.setLevel(level)

        console = logging.StreamHandler()
        console.setLevel(level)

        if formatstring:
            formatter = logging.Formatter(formatstring, datefmt='%Y-%m-%dT%H:%M:%S%z')
        else:
            if timestamp:
                formatter = logging.Formatter("%(asctime)s %(levelname)s\t%(process)d\t%(name)s\t%(filename)s\t%(lineno)d\t%(message)s", datefmt='%Y-%m-%dT%H:%M:%S%z')
            else:
                formatter = logging.Formatter("%(levelname)s\t%(process)d\t%(name)s\t%(filename)s\t%(lineno)d\t%(message)s")
            
        # tell the handler to use this format
        console.setFormatter(formatter)

        # add the handler to the root logger
        logger.addHandler(console)
        return logger
    else:
        #logging.getLogger(name).info(f'There already is a logger installed for {name}.')
        return logging.getLogger(name)


## Various functions

In [ ]:
#| export

def dict_structure(the_dict:dict, indent='    ') -> str:
    # current level
    current = []
    for key, item in the_dict.items():
        current.append(f"{indent}{str(key):16}:\t{str(type(item))}")
        if isinstance(item, dict) and item:
            current.append(dict_structure(item, f"{indent}    "))
    return "\n".join(current)


In [ ]:
print(dict_structure({
    "key_1": "A string",
    "longerkey_2": [1,2,3],
    "key_3": dict(a=[], b="2", c=3)
}))

    key_1           :	<class 'str'>
    longerkey_2     :	<class 'list'>
    key_3           :	<class 'dict'>
        a               :	<class 'list'>
        b               :	<class 'str'>
        c               :	<class 'int'>


## Date & time


In [ ]:
#| export

@lru_cache(256)
def make_datetime(d):
    # use parser to convert from string representation
    if isinstance(d, str):
        return parser.parse(d)
    # ints and floats are asumed to be unix timestamps (seconds sinds 1/1/1970)
    elif isinstance(d, int):
        return datetime.datetime.fromtimestamp(d)
    elif isinstance(d, float):
        return datetime.datetime.fromtimestamp(d)
    # for other cases use the object's string cast and parse that
    else:
        return parser.parse(str(d))


@lru_cache(128)
def make_naive_utc(d):
    if d.tzinfo is None or d.tzinfo.utcoffset(d) is None:
        timezone = pytz.timezone("UTC")
        return timezone.localize(d)
    return d

@lru_cache(128)
def make_naive_tzaware(d, tz='UTC'):
    if d.tzinfo is None or d.tzinfo.utcoffset(d) is None:
        timezone = pytz.timezone(tz)
        return timezone.localize(d)
    return d

@lru_cache(128)
def unix_time(some_time):
    """Convert to unix (epoch) timestamp"""
    if isinstance(some_time, datetime64):
        return (
                    datetime64(some_time,"ms")
                        - datetime64('1970-01-01T00:00:00')
                ) / timedelta64(1, 's')
    elif isinstance(some_time, datetime.date):
        return some_time.timestamp()
    elif isinstance(some_time, float):
        return some_time
    elif isinstance(some_time, int):
        return some_time
    else:
        return make_datetime(some_time).timestamp()



In [ ]:
#| hide
import nbdev
nbdev.nbdev_export()